In [1]:
import requests
from bs4 import BeautifulSoup
import time
import datetime
import json
import os

class Cenital:
    
    myname = "Cenital"
    savefolder = "./dataCenital/"
    rawdata = "raw/"
    
    class Home:
    
        url = "https://www.cenital.com/"
        
        def buscarnotas():
            
            cenital = Cenital()
            cenital.gethome()
            listadenotas = cenital.home.getlistofnotes()
            Cenital.saveinfo(listadenotas)
            Cenital.removemetadata(listadenotas)
            return listadenotas

        def __init__ (self):
            self.page = requests.get(self.url)
            if self.page.status_code != 200:
                print (self.page.status_code)
            else:
                print ("Home de " + Cenital.myname + " descargado con exito.")
            self.bpage = BeautifulSoup(self.page.content, 'html.parser')

        def printpretty(self):
            print(self.bpage.prettify())
            
        def getlistofnotes(self):
            info = Cenital.generateinfo("-notasHome")
            # Buscamos con el class card-title todas las notas porque es lo unico que idententifica a todas las notas del home
            titulos = self.bpage.find_all(attrs={"class":"card-title"})
            for titulo in titulos:
                vinculo = titulo.find_all("a")
                assert len (vinculo) == 1, "error, se encontro mas de un titulo por nota"
                info [vinculo[0]['href']] = {}
                info [vinculo[0]['href']]["Url"] = Cenital.Home.url + vinculo[0]['href']
                info [vinculo[0]['href']]["Titulo"] = vinculo[0].string
            return info
            
    class Noticia:
        
        def __init__ (self,url):
            self.url = url
            self.id = self.url.split("/")[-1]
            print (self.id)
            self.page = requests.get(self.url)
            if self.page.status_code != 200:
                print (self.page.status_code)
            else:
                print ("Nota " + self.url + " descargada con exito.")
            self.bpage = BeautifulSoup(self.page.content, 'html.parser')

        def printpretty(self):
            print(self.bpage.prettify())
        
        def generarinfo(self):
            self.info = Cenital.generateinfo("notaid-"+self.id,fecha=False)
            self.datosprevios = False
            # Aca hay que recopilar la info como si no hubiera datos previos y despues comparar en caso de que haya
            self.info["Titulo"] = self.bpage.find_all("title")[0].string
            self.buscardatosprevios()
            Cenital.saveinfo(self.info)
            return self.info
            
        def buscardatosprevios(self):
            if os.path.exists(self.info["metadata"]["filename"]):
                self.datosprevios = True
                with open(self.info["metadata"]["filename"]) as file:  
                    self.infoprevia = json.load(file)
                
                
    def gethome(self):
        self.home = self.Home()
        
    def generateinfo (name,fecha=True):
        info = {}
        info["metadata"] = {}
        info["metadata"]["timestampofsearch"] = datetime.datetime.now().timestamp()
        info["metadata"]["timeofsearch"] = datetime.datetime.fromtimestamp(info["metadata"]["timestampofsearch"]).isoformat()
        if fecha:
            info["metadata"]["filename"] = Cenital.savefolder + Cenital.rawdata + datetime.datetime.fromtimestamp(info["metadata"]["timestampofsearch"]).strftime("%Y-%m-%d-%H-%M") + name + ".json"
        else:
            info["metadata"]["filename"] = Cenital.savefolder + Cenital.rawdata + name + ".json"
        return info
        
    def removemetadata(info):
        del info["metadata"]
        
    def saveinfo(info):
        with open(info["metadata"]["filename"], 'w') as f:
            json.dump(info, f)
            
    
        
        
    # Inicializa las carpetas si no existen
    if not os.path.exists(savefolder):
        os.makedirs(savefolder)
    if not os.path.exists(savefolder+rawdata):
        os.makedirs(savefolder+rawdata)
    
    

In [3]:
listadenotas = Cenital.Home.buscarnotas()
print (listadenotas)

Home de Cenital descargado con exito.
{'2019/06/21/iran-y-estados-unidos-caminar-al-borde-del-abismo/63805': {'Url': 'https://www.cenital.com/2019/06/21/iran-y-estados-unidos-caminar-al-borde-del-abismo/63805', 'Titulo': 'Irán y Estados Unidos: caminar al borde del abismo'}, '2019/06/20/que-significa-que-los-gobernadores-vayan-con-lista-corta-/63795': {'Url': 'https://www.cenital.com/2019/06/20/que-significa-que-los-gobernadores-vayan-con-lista-corta-/63795', 'Titulo': ' ¿Qué significa que los gobernadores vayan con "lista corta"?'}, '2019/06/21/el-oficialismo-definio-sus-listas-para-la-ciudad-y-provincia-de-buenos-aires/63802': {'Url': 'https://www.cenital.com/2019/06/21/el-oficialismo-definio-sus-listas-para-la-ciudad-y-provincia-de-buenos-aires/63802', 'Titulo': ' El oficialismo definió sus listas para la Ciudad y Provincia de Buenos Aires'}, '2019/06/21/un-año-con-el-fmi/63800': {'Url': 'https://www.cenital.com/2019/06/21/un-año-con-el-fmi/63800', 'Titulo': ' Un año con el FMI'}, '

In [34]:
"""
listadenotas = Cenital.Home.buscarnotas()
for nota in list(listadenotas.values()):
    print (nota["Titulo"])
"""
noticia = Cenital.Noticia(list( Cenital.Home.buscarnotas().values())[1]["Url"])
noticia.bpage.find_all("title")

Home de Cenital descargado con exito.
63776
Nota https://www.cenital.com/2019/06/16/la-soga-que-une-dolores-con-comodoro-py/63776 descargada con exito.


[<title>La soga que une Dolores con Comodoro Py</title>]

In [35]:
noticia.bpage.find_all("title")[0].string

'La soga que une Dolores con Comodoro Py'

In [36]:
print (noticia.generarinfo())



cucu
{'metadata': {'timestampofsearch': 1560724008.685024, 'timeofsearch': '2019-06-16T19:26:48.685024', 'filename': './dataCenital/raw/notaid-63776.json'}, 'Titulo': 'La soga que une Dolores con Comodoro Py'}


In [63]:
for titulo in titulos:
    vinculo = titulo.find_all("a")
    assert len (vinculo) == 1, "error, se encontro mas de un titulo por nota"
    print (vinculo[0].string)
    print (vinculo[0]['href'])

¿Cómo funciona el FMI por dentro?
2019/06/15/como-funciona-el-fmi-por-dentro-/63773
 De dónde vienen los dólares
2019/06/15/de-donde-vienen-los-dolares/63771
 ¿Inflación o inflaciones?
2019/06/14/inflacion-o-inflaciones-/63770
 Qué dicen los balances de Mercado Libre
2019/06/14/que-dicen-los-balances-de-mercado-libre/63766
 Para Pichetto, "va a haber mucho peronismo detrás del presidente Macri"
2019/06/14/para-pichetto--va-a-haber-mucho-peronismo-detras-del-presidente-macri/63768
 Se viene otro superdomingo electoral
2019/06/14/se-viene-otro-superdomingo-electoral/63765
 La Justicia avaló la decisión de Macri de no votar para legisladores del Mercosur
2019/06/14/la-justicia-avalo-la-decision-de-macri-de-no-votar-para-legisladores-del-mercosur/63767
 Estados Unidos refuerza la acusación contra Irán con la publicación de un video
2019/06/14/estados-unidos-refuerza-la-acusacion-contra-iran-con-la-publicacion-de-un-video/63763
Delivery de gobernabilidad
2019/06/13/delivery-de-gobernabilida

In [55]:
titulos[0]

<h1 class="card-title titulo titulo-lg"><span class="amarillo font-16 hidden"> 15/06/2019 18:06:00</span><a href="2019/06/15/como-funciona-el-fmi-por-dentro-/63773">¿Cómo funciona el FMI por dentro?</a></h1>